In [1]:
using ANN

type Antenna
    x::Float64
    y::Float64
end

antennas = Antenna[]
push!(antennas, Antenna(0, 10))
push!(antennas, Antenna(0, -10))
push!(antennas, Antenna(10, 0))

fi(a, x, y) = 1 / ((x-antennas[a].x)^2 + (y-antennas[a].y)^2 + 1)
flog(a, x, y) = 5-log((x-antennas[a].x)^2 + (y-antennas[a].y + .1)^2)/2
# vectorized
flogv(a, x, y) = 5-log((x.-antennas[a].x).^2 .+ (y.-antennas[a].y + .1).^2)/2

N_set = 500
X_test = ones(N_set, 3)

# Kinda like a C struct
type StandardScalar
    mean::Vector{Float64}
    std::Vector{Float64}
end

# Helper function to initialize an empty scalar
function StandardScalar()
    StandardScalar(Array(Float64,0),Array(Float64,0))
end

# Compute mean and standard deviation of each column
function fit_std_scalar!(std_scalar::StandardScalar,X::Matrix{Float64})
    n_rows, n_cols = size(X_test)
    std_scalar.std = zeros(n_cols)
    std_scalar.mean = zeros(n_cols)
    # for loops are fast again!
    for i = 1:n_cols
        std_scalar.mean[i] = mean(X[:,i])
        std_scalar.std[i] = std(X[:,i])
    end
end

function transform(std_scalar::StandardScalar,X::Matrix{Float64})
    (X .- std_scalar.mean') ./ std_scalar.std' # broadcasting fu
end

# fit and transform in one function
function fit_transform!(std_scalar::StandardScalar,X::Matrix{Float64})
    fit_std_scalar!(std_scalar,X)
    transform(std_scalar,X)
end

tg(x, y) = convert(Int64, ((abs(x)<20) && (abs(y)<20)))

function tg2(x, y)
    if ((abs(x)<20) && (abs(y)<20))
        if (x>=0)
            if (y>=0)
                return 1
            else
                return 2
            end
        else
            if (y>=0)
                return 3
            else
                return 4
            end
        end
    else
        return 0
    end
end

function gen_datain(x, y)
    len = length(x)
    rss1 = []
    rss2 = []
    rss3 = []
    for i in 1:len
        rss1 = [flog(1, x[i], y[i]) for i in 1:len]
        rss2 = [flog(2, x[i], y[i]) for i in 1:len]
        rss3 = [flog(3, x[i], y[i]) for i in 1:len]
    end
    return rss1, rss2, rss3
end

function gen_datain2(x, y)
    len = length(x)
    rss1 = flogv(1, x, y)
    rss2 = flogv(2, x, y)
    rss3 = flogv(3, x, y)
    return rss1, rss2, rss3
end

function gen_dataout(x, y)
    len = length(x)
    out = []
    for i in 1:len
        out = [tg2(x[i], y[i]) for i in 1:len]
    end
    return out
end

std_scalar = StandardScalar()

N_train = 5000
# X_train = 100*rand(N_train, 2)-50

println("generating training data")
Rxy = 100*rand(2, N_train)-50
rss1, rss2, rss3 = gen_datain2(Rxy[1,:], Rxy[2,:])

X_train = [rss1 rss2 rss3]
y_train = gen_dataout(Rxy[1,:], Rxy[2,:])

X_train = fit_transform!(std_scalar,X_train)

println("training network")
ann = ArtificialNeuralNetwork(64)
fit!(ann,X_train,y_train,epochs=100,alpha=0.1,lambda=1e-5)


generating training data
training network


In [2]:
println("generating test data")

N_test = 5000
# X_test = 100*rand(N_test, 2)-50
Rxy = 100*rand(2, N_test)-50
rss1, rss2, rss3 = gen_datain2(Rxy[1,:], Rxy[2,:])

X_test = [rss1 rss2 rss3]
y_test = gen_dataout(Rxy[1,:], Rxy[2,:])

X_test = transform(std_scalar,X_test)

println("predicting data")

y_proba = ANN.predict(ann,X_test)

y_pred = Array(Int64,length(y_test))

for i in 1:length(y_test)
    # must translate class index to label
    y_pred[i] = ann.classes[indmax(y_proba[i,:])]
end

println("Prediction accuracy: ", mean(y_pred .== y_test))

function confusion_matrix(y_true::Array{Int64,1},y_pred::Array{Int64,1})
    # Generate confusion matrix
    cm = zeros(Int64,5,5)

    for i in 1:length(y_test)
        # translate label to index
        true_class = y_test[i]+1
        pred_class = y_pred[i]+1
        # pred class is the row, true class is the column
        cm[pred_class,true_class] += 1
    end
    cm
end

confusion_matrix(y_test,y_pred)

generating test data
predicting data
Prediction accuracy: 0.9826


5×5 Array{Int64,2}:
 4208   10   10    3    0
    2  167    2    2    0
    0    5  171    0    0
   18    3    0  183    0
   20    0   12    0  184

In [ ]:
using PyPlot

xx=zeros(Float64,length(y_test),1)
yy=zeros(Float64,length(y_test),1)
color=zeros(Int64,length(y_test),1)
for i in 1:length(y_test)
    xx[i] = Rxy[1,i]
    yy[i] = Rxy[2,i]
    color[i] = y_pred[i]
end

scatter(xx, yy, c=color)